In [37]:
#import common libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import plotly to make interactive plots
import plotly.express as px
import plotly.graph_objs as go

from plotly.offline import iplot, init_notebook_mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings("ignore")

In [38]:
#load files
country=pd.read_csv('C:\\Users\\LAP13393-local\\OneDrive - VNG Corporation\\Desktop\\Data analyst\\Vaccine Covid\\country_vaccinations.csv')
manu=pd.read_csv('C:\\Users\\LAP13393-local\\OneDrive - VNG Corporation\\Desktop\\Data analyst\\Vaccine Covid\\country_vaccinations_by_manufacturer.csv')

In [39]:
#process data
manu['date']=pd.to_datetime(manu['date'])
country['date']=pd.to_datetime(country['date'])

In [42]:
##FILE MANU
#Process data to count new vaccinations by day from cumulative total vaccinations 
manu['new']=0
for i in range(len(manu)-1):
    if((manu.loc[i+1,'vaccine']==manu.loc[i,'vaccine'])&(manu.loc[i+1,'location']==manu.loc[i+1,'location'])):
        manu.loc[i,'new']=manu.loc[i,'total_vaccinations']-manu.loc[i+1,'total_vaccinations']
    else:
        manu.loc[i,'new']=manu.loc[i,'total_vaccinations']

#Count the number of vaccinations by vaccine
manu_vaccine=manu.groupby(['vaccine']).agg({'new':sum})
manu_vaccine.reset_index(inplace=True)
manu_vaccine.rename({'new':'Total vaccinations'},axis=1,inplace=True)
fig=px.bar(manu_vaccine,x='vaccine',y='Total vaccinations',title='Total vaccinations by vaccine in Europe, US, Japan, Uruguay from January to July 2021')
fig.show()

In [51]:

#Select European countries to investigate
non_european = ['Chile', 'United States', 'Uruguay','Japan']
manu_europe = manu.query('location not in @non_european')
#Process data to count new vaccinations by day from cumulative total vaccinations
manu_europe=manu_europe.sort_values(by=['location','vaccine','date'], ascending=[True,True,False])
manu_europe=manu_europe.reset_index(drop=True)
manu_europe['new']=0
for i in range(len(manu_europe)-1):
    if((manu_europe.loc[i+1,'vaccine']==manu_europe.loc[i,'vaccine'])&(manu_europe.loc[i+1,'location']==manu_europe.loc[i+1,'location'])):
        manu_europe.loc[i,'new']=manu_europe.loc[i,'total_vaccinations']-manu_europe.loc[i+1,'total_vaccinations']
    else:
        manu_europe.loc[i,'new']=manu_europe.loc[i,'total_vaccinations']


In [53]:
#Calculate cumulative vaccinations by vaccines in Europe countries
vaccines_new=manu_europe[['date','vaccine','new']]
df=vaccines_new[['date','vaccine']]
df.drop_duplicates(inplace=True)
df.sort_values(by=['vaccine','date'],ascending=[True,False]).reset_index(drop=True)
df['before']=0
for i,row in df.iterrows():
    total_before=vaccines_new[(vaccines_new.vaccine == row.vaccine)&(vaccines_new.date<=row.date)]['new'].sum()
    df.loc[i,'before']=total_before
#Plot number of vaccinations by vaccines in Europes countries
df=df.pivot(index='date',columns='vaccine',values='before')
plt.figure(figsize=(8,10))
df.iplot(mode = 'lines', xTitle = 'Date', yTitle = 'Total vaccinations', title = 'Cumulative vaccinations in Europe by vaccine')

<Figure size 576x720 with 0 Axes>

In [55]:
#Plot for USA
manu_usa=manu.loc[manu['location']=='United States']
fig=px.line(manu_usa,x='date',y='total_vaccinations',color='vaccine',title='Cumulative vaccinations by vaccines in USA')
fig.show()

In [62]:
#Plot for Germany
manu_germany=manu.loc[manu['location']=='Germany']
fig=px.line(manu_germany,x='date',y='total_vaccinations',color='vaccine',title='Cumulative vaccinations by vaccines in Germany')
fig.show()

In [63]:
manu_netherlands=manu.loc[manu['location']=='Netherlands']
fig=px.line(manu_netherlands,x='date',y='total_vaccinations',color='vaccine',title='Cumulative vaccinations by vaccines in Netherlands')
fig.show()

In [4]:
##FILE COUNTRY
#select some countries to analyze
selected_countries=['United States','United Kingdom','New Zealand','Australia','India','Indonesia','Italy','Germany','China','Japan','Vietnam']
country_s = country.query('country in @selected_countries')
#Show which vaccines used in each countries
country_s_vaccine=country_s[['country','date','vaccines']]
country_s_vaccine=country_s_vaccine.sort_values(by=['country','date'],ascending=[True,False])
country_s_vaccine=country_s_vaccine.drop_duplicates(subset='country',keep='first')

In [10]:
country_s_vaccine

,country,date,vaccines
1620,Australia,2021-07-17,"Oxford/AstraZeneca, Pfizer/BioNTech"
5807,China,2021-07-17,"CanSino, Sinopharm/Beijing, Sinopharm/Wuhan, S..."
10663,Germany,2021-07-16,"Johnson&Johnson, Moderna, Oxford/AstraZeneca, ..."
12999,India,2021-07-16,"Covaxin, Oxford/AstraZeneca, Sputnik V"
13186,Indonesia,2021-07-17,"Oxford/AstraZeneca, Sinovac"
14259,Italy,2021-07-17,"Johnson&Johnson, Moderna, Oxford/AstraZeneca, ..."
14538,Japan,2021-07-15,"Moderna, Pfizer/BioNTech"
20400,New Zealand,2021-07-13,Pfizer/BioNTech
30047,United Kingdom,2021-07-16,"Moderna, Oxford/AstraZeneca, Pfizer/BioNTech"
30257,United States,2021-07-17,"Johnson&Johnson, Moderna, Pfizer/BioNTech"


In [11]:
#Extract metrics 'total vaccinations,total vaccinations per hundred,daily vaccinations per million' to analyze
total_vaccinations=country_s[['country','date','total_vaccinations']]
total_vaccinations_per_hundred=country_s[['country','date','total_vaccinations_per_hundred']]
daily_vaccinations=country_s[['country','date','daily_vaccinations_per_million']]
#Turn to pivot tables
total_vaccinations=total_vaccinations.pivot(index='date',columns='country',values='total_vaccinations')
total_vaccinations_per_hundred=total_vaccinations_per_hundred.pivot(index='date',columns='country',values='total_vaccinations_per_hundred')
daily_vaccinations=daily_vaccinations.pivot(index='date',columns='country',values='daily_vaccinations_per_million')

In [15]:
total_vaccinations

country,Australia,China,Germany,India,Indonesia,Italy,Japan,New Zealand,United Kingdom,United States,Vietnam
date,,,,,,,,,,,
2020-12-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86465.0,NaN,NaN
2020-12-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-15,NaN,1.500000e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2021-07-13,9304026.0,1.402019e+09,83308249.0,387697935.0,51433567.0,58837850.0,65028817.0,1404343.0,81192857.0,334942236.0,4079066.0
2021-07-14,9469145.0,1.414609e+09,84268098.0,391340491.0,54520987.0,59398005.0,65985026.0,NaN,81438892.0,335487779.0,4146767.0
2021-07-15,9631807.0,1.426347e+09,85014210.0,395343767.0,55819781.0,59996509.0,66714528.0,NaN,81702466.0,336054953.0,4185623.0


In [65]:
#Plot total vaccinations,total vaccinations per hundred, daily vaccinations per million in selected countries
plt.figure(figsize=(6,6))
total_vaccinations.iplot(mode = 'lines', xTitle = 'Date', yTitle = 'Total vaccinations', title = 'Cumulative vaccinations in some countries up to date')
total_vaccinations_per_hundred.iplot(mode = 'lines', xTitle = 'Date', yTitle = 'Total vaccinations per hundred', title = 'Cumulative total vaccinations per hundred in some countries up to date')
daily_vaccinations.iplot(mode = 'lines', xTitle = 'Date', yTitle = 'Daily vaccinations per million', title = 'Daily vaccinations per million in some countries up to date')


<Figure size 432x432 with 0 Axes>

In [66]:
#Extract some developed countries to plot
total_vaccinations_developed=total_vaccinations[['United States','United Kingdom','Germany','Australia','New Zealand','Italy']]
total_vaccinations_per_hundred_developed=total_vaccinations_per_hundred[['United States','United Kingdom','Germany','Australia','New Zealand','Italy']]
daily_vaccinations_developed=daily_vaccinations[['United States','United Kingdom','Germany','Australia','New Zealand','Italy']]
#Plot metrics for developed countries
total_vaccinations_developed.iplot(mode = 'lines', xTitle = 'Date', yTitle = 'Total vaccinations', title = 'Cumulative total vaccinations in some developed countries up to date')
total_vaccinations_per_hundred_developed.iplot(mode = 'lines', xTitle = 'Date', yTitle = 'Total vaccinations per hundred', title = 'Cumulative total vaccinations per hundred in some developed countries up to date')
daily_vaccinations_developed.iplot(mode = 'lines', xTitle = 'Date', yTitle = 'Daily vaccinations per million', title = 'Daily vaccinations per million in some  developed countries up to date')

In [67]:
#Extract Asian coutries to plot
plt.figure(figsize=(4,4))
total_vaccinations_Asian=total_vaccinations[['Vietnam','India','China','Japan','Indonesia']]
total_vaccinations_per_hundred_Asian=total_vaccinations_per_hundred[['Vietnam','India','China','Japan','Indonesia']]
daily_vaccinations_Asian=daily_vaccinations[['Vietnam','India','China','Japan','Indonesia']]
#Plot metrics for Asian countries
total_vaccinations_Asian.iplot(mode = 'lines', xTitle = 'Date', yTitle = 'Total vaccinations', title = 'Total vaccinations in some Asian countries up to date')
total_vaccinations_per_hundred_Asian.iplot(mode = 'lines', xTitle = 'Date', yTitle = 'Total vaccinations per hundred', title = 'Total vaccinations per hundred in some Asian countries up to date')
daily_vaccinations_Asian.iplot(mode = 'lines', xTitle = 'Date', yTitle = 'Daily vaccinations per million', title = 'Daily vaccinations per million in some Asian countries up to date')

<Figure size 288x288 with 0 Axes>